In [1]:
from time import time
from pathlib import Path
from tqdm import tqdm

from itertools import product

import numpy as np

data_dir = Path("data")
data_dir.mkdir(parents=True, exist_ok=True)

rng_seed = 42

print(f"Data will be saved to {data_dir.resolve()}")
print(f"Random seed is {rng_seed}")

Data will be saved to /Users/Tonni/Desktop/master-code/neural-quantum-tomo/case_studies/w_aug/data
Random seed is 42


In [2]:
# if we do just sampling, we actually don't need any eigenvalues

class PauliOperator:
    def __init__(self, matrix: np.ndarray, eigenvectors: list[np.ndarray]):
        """
        Bundles the relevant information of a Pauli operator into a single object.

        :param matrix: The matrix representation of the Pauli operator, shape (2, 2).
        :param eigenvectors: The eigenvectors of the Pauli operator, shape (2, 2).
        """
        self.matrix = matrix
        self.eigenvectors = eigenvectors


def construct_measurement_basis(pauli_ops: list[PauliOperator]) -> list[np.ndarray]:
    """
    Constructs the full tensor product basis for a given list of Pauli operators.

    :param pauli_ops: List of Pauli operators in order of qubit indices.

    :return: List basis vectors to project onto, shape (2^num_qubits, 2^num_qubits).
    """
    measurement_basis = []

    # we simply build up all possible outcomes like 000, 001, 010, ... 111
    outcome_bitstrings = list(product([0, 1], repeat=len(pauli_ops)))
    for outcome_bitstring in tqdm(outcome_bitstrings, desc="Constructing measurement basis"):
        multi_qubit_eigenvector = None

        # now we split up the bitstring and zip each qubit outcome with the corresponding Pauli operator
        for pauli_op, outcome_bit in zip(pauli_ops, outcome_bitstring):
            # we can directly misuse the outcome bit as index for accessing the state vector
            eigenvector = pauli_op.eigenvectors[outcome_bit]

            if multi_qubit_eigenvector is None:
                multi_qubit_eigenvector = eigenvector
                continue

            # we build up the multi-qubit eigenvector by taking the tensor product of the single-qubit eigenvectors
            multi_qubit_eigenvector = np.kron(multi_qubit_eigenvector, eigenvector)

        # each outcome has its corresponding measurement basis vector
        measurement_basis.append(multi_qubit_eigenvector)

    return measurement_basis


def sample_state(state_vec: np.ndarray, meas_dirs: list[str], num_samples: int = 1000, rng=None) -> np.ndarray:
    """
    Samples `num_samples` bitstrings from the state for the given measurement directions.

    :param state_vec: The state vector to sample from, shape (2^num_qubits,).
    :param meas_dirs: A description of the measurement directions, e.g. ['Z', 'X', 'Y'], shape (num_qubits,).
    :param num_samples: The number of samples to draw.
    :param rng: Optional random number generator. If None, a new default_rng will be created.

    :return: A list of sampled bitstrings, shape (num_samples, num_qubits).
    """
    rng = np.random.default_rng() if rng is None else rng

    # looking up the pauli operators, ignoring key errors at this point
    pauli_ops = [pauli_operators[md] for md in meas_dirs]

    basis_vecs = construct_measurement_basis(pauli_ops)

    start_time = time()

    # as soon as we have the basis vectors, we can apply the born rule spitting out the probabilities
    probs = [abs(np.vdot(v, state_vec))**2 for v in basis_vecs]
    probs = np.array(probs)
    probs /= probs.sum()  # just in case

    chosen_basis_indices = rng.choice(len(probs), size=num_samples, p=probs)
    sampled_bitstrings = [list(map(int, f"{basis_idx:0{len(pauli_ops)}b}")) for basis_idx in chosen_basis_indices]

    print(f"Sampled {num_samples} bitstrings in {time() - start_time:.2f} seconds.")
    return np.array(sampled_bitstrings, dtype=np.uint8)

In [3]:
def format_bytes(num_bytes):
    """Utility function to format bytes into KB, MB, GB, etc."""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if num_bytes < 1024.0:
            return f"{num_bytes:.2f} {unit}"
        num_bytes /= 1024.0
    return f"{num_bytes:.2f} PB"


# construct the state

num_qubits = 10

state_dim = 1 << num_qubits                     # bit shifting a ...0001 bitstring is the same as 2**a
w_aug = np.zeros(state_dim, dtype=complex)      # empty state vector

# since the W state has only non-zero amplitudes for one-hot states, we only need num_qubits random phases
rng = np.random.default_rng(42)
thetas = rng.uniform(0, 2*np.pi, size=num_qubits)

for j in range(num_qubits):
    idx = 1 << (num_qubits - 1 - j)              # find indexing mask via bit shifting

    # apply the phase to the corresponding amplitude coefficient
    w_aug[idx] = np.exp(1j * thetas[j]) / np.sqrt(num_qubits)


print(f"Size of state vector in memory: {format_bytes(w_aug.nbytes)} \n")

for i in range(10):
    print(f"{i:0{num_qubits}b}: {w_aug[i]:.2f} + {w_aug[i].imag:.2f}j")

Size of state vector in memory: 16.00 KB 

0000000000: 0.00+0.00j + 0.00j
0000000001: -0.30+0.10j + 0.10j
0000000010: 0.22+0.23j + 0.23j
0000000011: 0.00+0.00j + 0.00j
0000000100: 0.07-0.31j + -0.31j
0000000101: 0.00+0.00j + 0.00j
0000000110: 0.00+0.00j + 0.00j
0000000111: 0.00+0.00j + 0.00j
0000001000: 0.02-0.32j + -0.32j
0000001001: 0.00+0.00j + 0.00j


In [4]:
pauli_operators = {
    'I': PauliOperator(np.eye(2), [np.array([1, 0]), np.array([0, 1])]),
    'X': PauliOperator(np.array([[0, 1], [1, 0]]), [np.array([1,  1]) / np.sqrt(2), np.array([1, -1]) / np.sqrt(2)]),
    'Y': PauliOperator(np.array([[0, -1j], [1j, 0]]), [np.array([1,  1j]) / np.sqrt(2), np.array([1, -1j]) / np.sqrt(2)]),
    'Z': PauliOperator(np.array([[1, 0], [0, -1]]), [np.array([1, 0]), np.array([0, 1])])
}

In [5]:
measurement_dirs = []

# all Z
measurement_dirs.append(['Z'] * num_qubits)

# sliding XX window
for i in range(num_qubits - 1):
    basis_list = ['Z'] * num_qubits
    basis_list[i] = 'X'
    basis_list[i+1] = 'X'
    measurement_dirs.append(basis_list)

# sliding XY window
for i in range(num_qubits - 1):
    basis_list = ['Z'] * num_qubits
    basis_list[i] = 'X'
    basis_list[i+1] = 'Y'
    measurement_dirs.append(basis_list)


for i, basis in enumerate(measurement_dirs):
    print(f"Basis {i:2d}: {''.join(basis)}")

Basis  0: ZZZZZZZZZZ
Basis  1: XXZZZZZZZZ
Basis  2: ZXXZZZZZZZ
Basis  3: ZZXXZZZZZZ
Basis  4: ZZZXXZZZZZ
Basis  5: ZZZZXXZZZZ
Basis  6: ZZZZZXXZZZ
Basis  7: ZZZZZZXXZZ
Basis  8: ZZZZZZZXXZ
Basis  9: ZZZZZZZZXX
Basis 10: XYZZZZZZZZ
Basis 11: ZXYZZZZZZZ
Basis 12: ZZXYZZZZZZ
Basis 13: ZZZXYZZZZZ
Basis 14: ZZZZXYZZZZ
Basis 15: ZZZZZXYZZZ
Basis 16: ZZZZZZXYZZ
Basis 17: ZZZZZZZXYZ
Basis 18: ZZZZZZZZXY


In [6]:
def format_measurement(bitstring, basis):
    result = []
    for bit, op in zip(bitstring, basis):
        if bit == 0 or bit == '0':
            result.append(op.upper())
        elif bit == 1 or bit == '1':
            result.append(op.lower())
        else:
            result.append('?')
    return ''.join(result)


samples_per_basis = 6400


for _, pauli_ops in enumerate(measurement_dirs):
    pauli_dirs_str = ''.join(pauli_ops)
    filename = data_dir / f"w_aug_{pauli_dirs_str}_{samples_per_basis}.txt"

    print(f"Measuring in basis: {pauli_dirs_str}")

    samples = sample_state(w_aug, pauli_ops, samples_per_basis, rng=rng)

    with open(filename, 'w') as f_out:
        for bitstring in samples:
            formatted = format_measurement(bitstring, pauli_ops)
            f_out.write(formatted + "\n")

    print(f"Stored {samples_per_basis} samples to {filename.name}.")

Measuring in basis: ZZZZZZZZZZ


Constructing measurement basis: 100%|██████████| 1024/1024 [00:00<00:00, 9508.18it/s]

Sampled 6400 bitstrings in 0.01 seconds.


Stored 6400 samples to w_aug_ZZZZZZZZZZ_6400.txt.
Measuring in basis: XXZZZZZZZZ


Constructing measurement basis: 100%|██████████| 1024/1024 [00:00<00:00, 10145.31it/s]


...

Sampled 6400 bitstrings in 0.02 seconds.
Stored 6400 samples to w_aug_ZZZZZZXYZZ_6400.txt.
Measuring in basis: ZZZZZZZXYZ


Constructing measurement basis: 100%|██████████| 1024/1024 [00:00<00:00, 10359.95it/s]


Sampled 6400 bitstrings in 0.01 seconds.
Stored 6400 samples to w_aug_ZZZZZZZXYZ_6400.txt.
Measuring in basis: ZZZZZZZZXY


Constructing measurement basis: 100%|██████████| 1024/1024 [00:00<00:00, 10164.45it/s]


Sampled 6400 bitstrings in 0.01 seconds.
Stored 6400 samples to w_aug_ZZZZZZZZXY_6400.txt.
